In [13]:
import pandas as pd
df = pd.read_excel(f'data/单次出行数据.xlsx')
people_data = pd.read_csv(f'data/MaaS被调查者聚类加个人属性加态度.csv',encoding = 'gb2312')
people_data = people_data[['peopleID','week_bus','week_metro', 'week_bike', 'week_ebike', 'week_taxi']]
people_data.columns = ['Uid','week_bus','week_metro', 'week_bike', 'week_ebike', 'week_taxi']
df = pd.merge(df,people_data)


df = df[-df['cheekall']]

In [ ]:
#不支持MaaS的表述正向化
reject = [2,3,5,20]
for i in reject:
    df['at'+str(i)] = 6 - df['at'+str(i)]
first_choice = pd.get_dummies(df['nonmaas'])

first_choice.columns = ['first_car','first_taxi','first_pt']

df = pd.concat([df,first_choice],axis = 1)

departtime = pd.get_dummies(df['departtime'])
departtime.columns = ['morning','evening','normal','late']
df = pd.concat([df,departtime],axis = 1)

def get_big_penalty(r,column,av):
    if r[av] == 1:
        return r[column]
    else:
        return 100

##出行模式哑元化
df['week_bus'] = df['week_bus'].replace(to_replace=[1,2,3,4,5],value=[0,0,0,1,1])
df['week_metro'] = df['week_metro'].replace(to_replace=[1,2,3,4,5],value=[0,0,0,1,1])
df['week_bike'] = df['week_bike'].replace(to_replace=[1,2,3,4,5],value=[0,0,0,1,1])
df['week_ebike'] = df['week_ebike'].replace(to_replace=[1,2,3,4,5],value=[0,0,0,1,1])
df['week_taxi'] = df['week_taxi'].replace(to_replace=[1,2,3,4,5],value=[0,0,0,1,1])

df['A1ttimeCar'] = df.apply(lambda r:get_big_penalty(r,'A1ttimeCar','Carav'),axis = 1)
df['A1priceCar'] = df.apply(lambda r:get_big_penalty(r,'A1priceCar','Carav'),axis = 1)

df['C1busrail'] = df.apply(lambda r:get_big_penalty(r,'C1busrail','PTav'),axis = 1)
df['C1ttimePT'] = df.apply(lambda r:get_big_penalty(r,'C1ttimePT','PTav'),axis = 1)
df['C1wtimePT'] = df.apply(lambda r:get_big_penalty(r,'C1wtimePT','PTav'),axis = 1)
df['C1distancePT_walk'] = df.apply(lambda r:get_big_penalty(r,'C1distancePT_walk','PTav'),axis = 1)
df['C1triptimePT'] = df.apply(lambda r:get_big_penalty(r,'C1triptimePT','PTav'),axis = 1)
df['C1pricePT'] = df.apply(lambda r:get_big_penalty(r,'C1pricePT','PTav'),axis = 1)

df['M1ttimerail'] = df.apply(lambda r:get_big_penalty(r,'M1ttimerail','Mav'),axis = 1)
df['M1ttime_bus'] = df.apply(lambda r:get_big_penalty(r,'M1ttime_bus','Mav'),axis = 1)
df['M1wtime'] = df.apply(lambda r:get_big_penalty(r,'M1wtime','Mav'),axis = 1)
df['M1distance_walk'] = df.apply(lambda r:get_big_penalty(r,'M1distance_walk','Mav'),axis = 1)
df['M1triptime'] = df.apply(lambda r:get_big_penalty(r,'M1triptime','Mav'),axis = 1)
df['M1price'] = df.apply(lambda r:get_big_penalty(r,'M1price','Mav'),axis = 1)

df['M2ttime_rail'] = df.apply(lambda r:get_big_penalty(r,'M2ttime_rail','Mav'),axis = 1)
df['M2ttime_bus'] = df.apply(lambda r:get_big_penalty(r,'M2ttime_bus','Mav'),axis = 1)
df['M2wtime'] = df.apply(lambda r:get_big_penalty(r,'M2wtime','Mav'),axis = 1)
df['M2distance_bike'] = df.apply(lambda r:get_big_penalty(r,'M2distance_bike','Mav'),axis = 1)
df['M2triptime'] = df.apply(lambda r:get_big_penalty(r,'M2triptime','Mav'),axis = 1)
df['M2price'] = df.apply(lambda r:get_big_penalty(r,'M2price','Mav'),axis = 1)

df['M3ttime_rail'] = df.apply(lambda r:get_big_penalty(r,'M3ttime_rail','PTav'),axis = 1)
df['M3ttime_taxi'] = df.apply(lambda r:get_big_penalty(r,'M3ttime_taxi','PTav'),axis = 1)
df['M3wtime'] = df.apply(lambda r:get_big_penalty(r,'M3wtime','PTav'),axis = 1)
df['M3triptime'] = df.apply(lambda r:get_big_penalty(r,'M3triptime','PTav'),axis = 1)
df['M3price'] = df.apply(lambda r:get_big_penalty(r,'M3price','PTav'),axis = 1)


def get_M1difference_time_prive(r):
    if r['first_car'] == 1:
        return (r['M1triptime'] - r['A1ttimeCar']), (r['M1price'] - r['A1priceCar'])
    if r['first_taxi'] == 1:
        return (r['M1triptime'] - r['B1triptime']), (r['M1price'] - r['B1priceTaxi'])
    if r['first_pt'] == 1:
        return (r['M1triptime'] - r['C1triptimePT']), (r['M1price'] - r['C1pricePT'])

def get_M2difference_time_prive(r):
    if r['first_car'] == 1:
        return (r['M2triptime'] - r['A1ttimeCar']), (r['M2price'] - r['A1priceCar'])
    if r['first_taxi'] == 1:
        return (r['M2triptime'] - r['B1triptime']), (r['M2price'] - r['B1priceTaxi'])
    if r['first_pt'] == 1:
        return (r['M2triptime'] - r['C1triptimePT']), (r['M2price'] - r['C1pricePT'])


def get_M3difference_time_prive(r):
    if r['first_car'] == 1:
        return (r['M3triptime'] - r['A1ttimeCar']), (r['M3price'] - r['A1priceCar'])
    if r['first_taxi'] == 1:
        return (r['M3triptime'] - r['B1triptime']), (r['M3price'] - r['B1priceTaxi'])
    if r['first_pt'] == 1:
        return (r['M3triptime'] - r['C1triptimePT']), (r['M3price'] - r['C1pricePT'])

def get_M4difference_time_prive(r):
    if r['first_car'] == 1:
        return (r['M4ttime'] - r['A1ttimeCar']), (r['M4price'] - r['A1priceCar'])
    if r['first_taxi'] == 1:
        return (r['M4ttime'] - r['B1triptime']), (r['M4price'] - r['B1priceTaxi'])
    if r['first_pt'] == 1:
        return (r['M4ttime'] - r['C1triptimePT']), (r['M4price'] - r['C1pricePT'])

def get_notransfer(r):
    if r['first_car'] == 1:
        return r['A1ttimeCar'], r['A1priceCar']
    if r['first_taxi'] == 1:
        return r['B1triptime'], r['B1priceTaxi']
    if r['first_pt'] == 1:
        return r['C1triptimePT'], r['C1pricePT']


df['no_triptime'], df['no_price'] = df.apply(get_notransfer,axis = 1).apply(lambda r:r[0]), df.apply(get_notransfer,axis = 1).apply(lambda r:r[1])


df['M1dif_triptime'], df['M1dif_price'] = df.apply(get_M1difference_time_prive,axis = 1).apply(lambda r:r[0]), df.apply(get_M1difference_time_prive,axis = 1).apply(lambda r:r[1])
df['M2dif_triptime'], df['M2dif_price'] = df.apply(get_M2difference_time_prive,axis = 1).apply(lambda r:r[0]), df.apply(get_M2difference_time_prive,axis = 1).apply(lambda r:r[1])
df['M3dif_triptime'], df['M3dif_price'] = df.apply(get_M3difference_time_prive,axis = 1).apply(lambda r:r[0]), df.apply(get_M3difference_time_prive,axis = 1).apply(lambda r:r[1])
df['M4dif_triptime'], df['M4dif_price'] = df.apply(get_M4difference_time_prive,axis = 1).apply(lambda r:r[0]), df.apply(get_M4difference_time_prive,axis = 1).apply(lambda r:r[1])


##轨道交通时间占比
df['M1metroratio'] = df['M1ttimerail']/df['M1triptime']
df['M2metroratio'] = df['M2ttime_rail']/df['M2triptime']
df['M3metroratio'] = df['M3ttime_rail']/df['M3triptime']


df['C1busrail'] = df['C1busrail'] - 1


##年龄哑变量化
df['age'] = df['age'].replace(to_replace=[1,2,3,4,5,6,7],value=[1,1,2,2,3,3,4])
age = pd.get_dummies(df['age'])
age.columns = ['age1','age2','age3','age4']
df = pd.concat([df,age],axis = 1)
##个人收入哑元化
df['income_m'] = df['income_m'].replace(to_replace=[1,2,3,4,5,6,7,8],value=[1,1,1,1,2,2,2,3])
income = pd.get_dummies(df['income_m'])
income.columns = ['income1','income2','income3']
df = pd.concat([df,income],axis = 1)
#个人收入，1W以下为0
df['income_m'] = df['income_m'].replace(to_replace=[1,2,3,4,5,6,7,8],value=[0,0,0,0,0,1,1,1])
#出行距离哑元化
distance = pd.get_dummies(df['distance'])
distance.columns = ['distance1','distance2','distance3','distance4','distance5']
df = pd.concat([df,distance],axis = 1)
#将1、2、3转换为0、1、2
df['gender'] = df['gender']-1
df['MainPurpose'] = df['MainPurpose']-1
df['member'] = df['member'].replace(to_replace=[1,2,3,4],value=[0,1,1,1])
df['DriveLicense'] = 2-df['DriveLicense']
df['DriveProf'] = 2-df['DriveProf']
df['bikeown'] = 2-df['bikeown']
df['MaasFamiliar'] = 2-df['MaasFamiliar']
#负数补0
df.loc[df['carnum']<0,'carnum'] = 0
df.loc[df['DriveLicense']<0,'DriveLicense'] = 0
##变量哑元化
#每月花费，150以下为0
# model_data['cost'] = model_data['cost'].replace(to_replace=[1,2,3,4,5,6],value=[0,0,1,1,1,1])
#年龄，34以下为0
df['age'] = df['age'].replace(to_replace=[1,2,3,4,5,6,7],value=[0,0,0,1,1,1,1])
#教育，本科以上为1
df['graduate'] = df['graduate'].replace(to_replace=[1,2,3,4,5,6],value=[1,1,1,0,0,0])


#家庭年收入，30W以下为0
df['income_y'] = df['income_y'].replace(to_replace=[1,2,3,4,5,6,7],value=[0,0,0,0,1,1,1])
#工作，出行少的工作为0
df['occupy'] = df['occupy'].replace(to_replace=[1,2,3,4,5,6,7,8,9,10,11],value=[0,0,0,0,0,0,0,1,1,1,1])

df['Carown'] = 2 - df['Carown']

df['TripsPerWeek'] = df['TripsPerWeek'].replace(to_replace=[1,2,3,4],value = [0,0,1,1])
df['DistanceWorkday'] = df['DistanceWorkday'].replace(to_replace=[1,2,3,4,5,6],value = [0,0,1,1,1,1])
df['DistanceWeekend'] = df['DistanceWeekend'].replace(to_replace=[1,2,3,4,5,6],value = [0,0,1,1,1,1])

df['DistanceWeekend'].value_counts()



df['maas'] = df['maas'].replace(to_replace=[1,2,3],value=[1,1,1])


dataframe = df.drop(columns=['cheek1', 'cheek2','cheekall'])

In [15]:
df = pd.read_csv(f'data/MaaS被调查者面对情景.csv',encoding = 'gb2312')
ls = []
for i in range(len(df)):
    if df.iloc[i,1] < 3:
        ques1 = df.iloc[i,2]
        ques2 = df.iloc[i,8]
        ques3 = df.iloc[i,14]
        ques4 = df.iloc[i,20]
        ques5 = df.iloc[i,26]
        anws1 = df.iloc[i,3]
        anws2 = df.iloc[i,9]
        anws3 = df.iloc[i,15]
        anws4 = df.iloc[i,21]
        anws5 = df.iloc[i,27]
        ls.append([df.iloc[i,0],1,ques1,anws1])
        ls.append([df.iloc[i,0],1,8+ques2,anws2])
        ls.append([df.iloc[i,0],1,16+ques3,anws3])
        ls.append([df.iloc[i,0],1,24+ques4,anws4])
        ls.append([df.iloc[i,0],1,32+ques5,anws5])
    elif df.iloc[i,1] < 5:
        ques1 = df.iloc[i,4]
        ques2 = df.iloc[i,10]
        ques3 = df.iloc[i,16]
        ques4 = df.iloc[i,22]
        ques5 = df.iloc[i,28]
        anws1 = df.iloc[i,5]
        anws2 = df.iloc[i,11]
        anws3 = df.iloc[i,17]
        anws4 = df.iloc[i,23]
        anws5 = df.iloc[i,29]
        ls.append([df.iloc[i,0],2,40+ques1,anws1])
        ls.append([df.iloc[i,0],2,48+ques2,anws2])
        ls.append([df.iloc[i,0],2,56+ques3,anws3])
        ls.append([df.iloc[i,0],2,64+ques4,anws4])
        ls.append([df.iloc[i,0],2,72+ques5,anws5])
    else:
        ques1 = df.iloc[i,6]
        ques2 = df.iloc[i,12]
        ques3 = df.iloc[i,18]
        ques4 = df.iloc[i,24]
        ques5 = df.iloc[i,30]
        anws1 = df.iloc[i,7]
        anws2 = df.iloc[i,13]
        anws3 = df.iloc[i,19]
        anws4 = df.iloc[i,25]
        anws5 = df.iloc[i,31]
        ls.append([df.iloc[i,0],3,80+ques1,anws1])
        ls.append([df.iloc[i,0],3,88+ques2,anws2])
        ls.append([df.iloc[i,0],3,96+ques3,anws3])
        ls.append([df.iloc[i,0],3,104+ques4,anws4])
        ls.append([df.iloc[i,0],3,112+ques5,anws5])
data = pd.DataFrame(ls,columns = ['index','week_taxi','bundle_index','results'])
question_bundle_data = pd.read_csv(f'data/question_bundle_data.csv',encoding = 'gb2312')
question_bundle_data = question_bundle_data.rename(columns = {'index':'bundle_index','class':'week_taxi'})
data = pd.merge(data,question_bundle_data,on = ['week_taxi','bundle_index'])
data = data.sort_values('index')
people_data = pd.read_csv(f'data/MaaS被调查者聚类加个人属性加态度.csv',encoding = 'gb2312')
##年龄哑变量化
people_data['age'] = people_data['age'].replace(to_replace=[1,2,3,4,5,6,7],value=[1,1,2,2,3,3,4])
age = pd.get_dummies(people_data['age'])
age.columns = ['age1','age2','age3','age4']
people_data = pd.concat([people_data,age],axis = 1)
##收入哑变量化
people_data['income_individual'] = people_data['income_individual'].replace(to_replace=[1,2,3,4,5,6,7,8],value=[1,1,1,2,2,2,2,3])
income = pd.get_dummies(people_data['income_individual'])
income.columns = ['income1','income2','income3']
people_data = pd.concat([people_data,income],axis = 1)
data = data.drop(columns=['week_taxi','bundle_index'])
data = data.rename(columns={'index':'peopleID'})
model_data = pd.merge(data,people_data,on = 'peopleID')
#将1、2、3转换为0、1、2
model_data['sex'] = model_data['sex']-1
model_data['travel_aim'] = model_data['travel_aim']-1
model_data['home_people'] = model_data['home_people']-1
model_data['license'] = 2-model_data['license']
model_data['drive'] = 2-model_data['drive']
model_data['e-bike'] = 2-model_data['e-bike']
#负数补0
model_data.loc[model_data['car_home']<0,'car_home'] = 0
model_data.loc[model_data['drive']<0,'drive'] = 0
##变量哑元化
#每月花费，150以下为0
# model_data['cost'] = model_data['cost'].replace(to_replace=[1,2,3,4,5,6],value=[0,0,1,1,1,1])
#年龄，34以下为0
model_data['age'] = model_data['age'].replace(to_replace=[1,2,3,4,5,6,7],value=[0,0,0,1,1,1,1])
#教育，本科以上为1
model_data['education'] = model_data['education'].replace(to_replace=[1,2,3,4,5,6],value=[1,1,1,0,0,0])
#个人收入，1W以下为0
model_data['income_individual'] = model_data['income_individual'].replace(to_replace=[1,2,3,4,5,6,7,8],value=[0,0,0,0,0,1,1,1])
#家庭年收入，30W以下为0
model_data['income_home'] = model_data['income_home'].replace(to_replace=[1,2,3,4,5,6,7],value=[0,0,0,0,1,1,1])
#工作，出行少的工作为0
model_data['occupy'] = model_data['occupy'].replace(to_replace=[1,2,3,4,5,6,7,8,9,10,11],value=[0,0,0,0,0,0,0,1,1,1,1])
##变量缩放
#共享电动车除10
model_data['ebike_12'] = model_data['ebike_12'] / 10
model_data['ebike_3'] = model_data['ebike_3'] / 10
model_data['ebike_4'] = model_data['ebike_4'] / 10
#出租车除100
model_data['taxi_12'] = model_data['taxi_12'] / 100
model_data['taxi_3'] = model_data['taxi_3'] / 100
model_data['taxi_4'] = model_data['taxi_4'] / 100
#价格除100
model_data['price1'] = model_data['price1'] / 100
model_data['price2'] = model_data['price2'] / 100
model_data['price3'] = model_data['price3'] / 100
model_data['price4'] = model_data['price4'] / 100
##列改名
model_data = model_data.rename(columns={'6a':'a6','6b':'b6','6c':'c6','6d':'d6','6e':'e6','6f':'f6','6g':'g6'})
model_data = model_data.rename(columns={'7a':'a7','7b':'b7','7c':'c7','7d':'d7'})
model_data = model_data.rename(columns={'e-bike':'e_bike'})

In [16]:
stage2_data = model_data[['peopleID', 'results','ebike_12', 'ebike_3', 'ebike_4', 'taxi_12',
       'taxi_3', 'taxi_4', 'price_12', 'price_3', 'price_4', 'price1',
       'price2', 'price3', 'price4','travel_num', 'travel_distance_work',
       'travel_distance_weekend', 'travel_aim', 'travel_people', 'a6', 'b6',
       'c6', 'd6', 'e6', 'f6', 'g6', 'a7', 'b7', 'c7', 'd7', 'cost', 'sex','education','income1', 'income2', 'income3','have_car',
       'license','e_bike','occupy', 'week_bus',
       'week_metro', 'week_bike', 'week_ebike', 'week_taxi', 'at1', 'at2',
       'at3', 'at4', 'at5', 'at6', 'at7', 'at8', 'at9', 'at10', 'at11', 'at12',
       'at13', 'at14', 'at15', 'at16', 'at17', 'at18', 'at19', 'at20', 'at21',
       'at22', 'at23', 'at24', 'at25','age1', 'age2',
       'age3', 'age4']]

In [17]:
stage1_data = dataframe[['maas','Uid','purpose', 'distance', 'B1wtimeTaxi','A1ttimeCar', 'A1priceCar',
       'B1ttimeTaxi', 'B1wtimeTaxi', 'B1triptime', 'B1priceTaxi', 'C1busrail',
       'C1ttimePT', 'C1wtimePT', 'C1distancePT_walk', 'C1triptimePT',
       'C1pricePT', 'M1ttimerail', 'M1ttime_bus', 'M1wtime', 'M1distance_walk',
       'M1triptime', 'M1price', 'M2ttime_rail', 'M2ttime_bus', 'M2wtime',
       'M2distance_bike', 'M2triptime', 'M2price', 'M3ttime_rail',
       'M3ttime_taxi', 'M3wtime', 'M3triptime', 'M3price', 'M4ttime',
       'M4price','first_car', 'first_taxi', 'first_pt','morning',
       'evening', 'normal', 'late','M1dif_triptime', 'M1dif_price', 'M2dif_triptime', 'M2dif_price',
       'M3dif_triptime', 'M3dif_price', 'M4dif_triptime', 'M4dif_price',
       'M1metroratio', 'M2metroratio', 'M3metroratio','distance1', 'distance2',
       'distance3', 'distance4', 'distance5','MaasFamiliar']]

In [18]:
stage1_data.rename(columns={'Uid':'peopleID'},inplace=True)

/tmp/ipykernel_4875/295337620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stage1_data.rename(columns={'Uid':'peopleID'},inplace=True)


In [19]:
all_stage_data = pd.merge(stage1_data,stage2_data,on = 'peopleID')

In [20]:
all_stage_data.columns[100:]

Index(['week_bus', 'week_metro', 'week_bike', 'week_ebike', 'week_taxi', 'at1',
       'at2', 'at3', 'at4', 'at5', 'at6', 'at7', 'at8', 'at9', 'at10', 'at11',
       'at12', 'at13', 'at14', 'at15', 'at16', 'at17', 'at18', 'at19', 'at20',
       'at21', 'at22', 'at23', 'at24', 'at25', 'age1', 'age2', 'age3', 'age4'],
      dtype='object')

In [21]:
all_stage_data.to_csv(f'data/最终模型数据.csv',index = False,encoding = 'gb2312')